# LLM Evaluation with MLflow Example Notebook

In this notebook, we will demonstrate how to evaluate various LLMs and RAG systems with MLflow, leveraging simple metrics such as toxicity, as well as LLM-judged metrics such as relevance, and even custom LLM-judged metrics such as professionalism.

We need to set our OpenAI API key, since we will be using GPT-4 for our LLM-judged metrics.

In order to set your private key safely, please be sure to either export your key through a command-line terminal for your current instance, or, for a permanent addition to all user-based sessions, configure your favored environment management configuration file (i.e., .bashrc, .zshrc) to have the following entry:

`OPENAI_API_KEY=<your openai API key>`

In [10]:
import openai
import pandas as pd

import mlflow

## Basic Question-Answering Evaluation

Create a test case of `inputs` that will be passed into the model and `ground_truth` which will be used to compare against the generated output from the model.

In [12]:
eval_df = pd.DataFrame(
    {
        "inputs": [
            "How does useEffect() work?",
            "What does the static keyword in a function mean?",
            "What does the 'finally' block in Python do?",
            "What is the difference between multiprocessing and multithreading?",
        ],
        "ground_truth": [
            "The useEffect() hook tells React that your component needs to do something after render. React will remember the function you passed (we'll refer to it as our 'effect'), and call it later after performing the DOM updates.",
            "Static members belongs to the class, rather than a specific instance. This means that only one instance of a static member exists, even if you create multiple objects of the class, or if you don't create any. It will be shared by all objects.",
            "'Finally' defines a block of code to run when the try... except...else block is final. The finally block will be executed no matter if the try block raises an error or not.",
            "Multithreading refers to the ability of a processor to execute multiple threads concurrently, where each thread runs a process. Whereas multiprocessing refers to the ability of a system to run multiple processors in parallel, where each processor can run one or more threads.",
        ],
    }
)

Create a simple OpenAI model that asks gpt-4o to answer the question in two sentences. Call `mlflow.evaluate()` with the model and evaluation dataframe. 

In [13]:
with mlflow.start_run() as run:
    system_prompt = "Answer the following question in two sentences"
    basic_qa_model = mlflow.openai.log_model(
        model="gpt-4o-mini",
        task=openai.chat.completions,
        name="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",  # specify which column corresponds to the expected output
        model_type="question-answering",  # model type indicates which metrics are relevant for this task
        evaluators="default",
    )
results.metrics

2024/01/04 11:22:24 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/01/04 11:22:24 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/01/04 11:22:26 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/01/04 11:22:26 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: token_count
2024/01/04 11:22:26 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: toxicity
2024/01/04 11:22:26 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: flesch_kincaid_grade_level
2024/01/04 11:22:26 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ari_grade_level
2024/01/04 11:22:26 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: exact_match


{'toxicity/v1/mean': 0.00020710023090941831,
 'toxicity/v1/variance': 3.7077160557159724e-09,
 'toxicity/v1/p90': 0.00027640480257105086,
 'toxicity/v1/ratio': 0.0,
 'flesch_kincaid_grade_level/v1/mean': 14.025,
 'flesch_kincaid_grade_level/v1/variance': 31.066875000000007,
 'flesch_kincaid_grade_level/v1/p90': 20.090000000000003,
 'ari_grade_level/v1/mean': 16.525,
 'ari_grade_level/v1/variance': 39.361875000000005,
 'ari_grade_level/v1/p90': 23.340000000000003,
 'exact_match/v1': 0.0}

Inspect the evaluation results table as a dataframe to see row-by-row metrics to further assess model performance

In [14]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,toxicity/v1/score,flesch_kincaid_grade_level/v1/score,ari_grade_level/v1/score
0,How does useEffect() work?,The useEffect() hook tells React that your com...,useEffect() is a hook in React that allows you...,51,0.000212,11.9,14.1
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The static keyword in a function means that th...,48,0.000145,10.7,12.9
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python is used to defin...,55,0.000304,9.9,11.8
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves using multiple proces...,36,0.000167,23.6,27.3


## LLM-judged correctness with OpenAI GPT-4

Construct an answer similarity metric using the `answer_similarity()` metric factory function.

In [15]:
from mlflow.metrics.genai import EvaluationExample, answer_similarity

# Create an example to describe what answer_similarity means like for this problem.
example = EvaluationExample(
    input="What is MLflow?",
    output="MLflow is an open-source platform for managing machine "
    "learning workflows, including experiment tracking, model packaging, "
    "versioning, and deployment, simplifying the ML lifecycle.",
    score=4,
    justification="The definition effectively explains what MLflow is "
    "its purpose, and its developer. It could be more concise for a 5-score.",
    grading_context={
        "targets": "MLflow is an open-source platform for managing "
        "the end-to-end machine learning (ML) lifecycle. It was developed by Databricks, "
        "a company that specializes in big data and machine learning solutions. MLflow is "
        "designed to address the challenges that data scientists and machine learning "
        "engineers face when developing, training, and deploying machine learning models."
    },
)

# Construct the metric using OpenAI GPT-4 as the judge
answer_similarity_metric = answer_similarity(model="openai:/gpt-4", examples=[example])

print(answer_similarity_metric)

EvaluationMetric(name=answer_similarity, greater_is_better=True, long_name=answer_similarity, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's answer_similarity based on the rubric
justification: Your reasoning about the model's answer_similarity score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called answer_similarity based on the input and output.
A definition of answer_similarity and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them be

Call `mlflow.evaluate()` again but with your new `answer_similarity_metric`

In [16]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[answer_similarity_metric],  # use the answer similarity metric created above
    )
results.metrics

2024/01/04 11:22:26 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.6.1.dev0, required: mlflow==2.9.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2024/01/04 11:22:26 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/01/04 11:22:27 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/01/04 11:22:28 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...


  0%|          | 0/1 [00:00<?, ?it/s]

2024/01/04 11:22:36 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: token_count
2024/01/04 11:22:36 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: toxicity
2024/01/04 11:22:36 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: flesch_kincaid_grade_level
2024/01/04 11:22:36 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ari_grade_level
2024/01/04 11:22:36 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: exact_match
2024/01/04 11:22:36 INFO mlflow.models.evaluation.default_evaluator: Evaluating metrics: answer_similarity


  0%|          | 0/4 [00:00<?, ?it/s]

{'toxicity/v1/mean': 0.00020082585615455173,
 'toxicity/v1/variance': 2.557426090397114e-09,
 'toxicity/v1/p90': 0.0002563037458457984,
 'toxicity/v1/ratio': 0.0,
 'flesch_kincaid_grade_level/v1/mean': 13.275000000000002,
 'flesch_kincaid_grade_level/v1/variance': 24.811875,
 'flesch_kincaid_grade_level/v1/p90': 18.930000000000003,
 'ari_grade_level/v1/mean': 15.35,
 'ari_grade_level/v1/variance': 32.522499999999994,
 'ari_grade_level/v1/p90': 21.88,
 'exact_match/v1': 0.0,
 'answer_similarity/v1/mean': 3.75,
 'answer_similarity/v1/variance': 1.1875,
 'answer_similarity/v1/p90': 4.7}

See the row-by-row LLM-judged answer similarity score and justifications

In [17]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,toxicity/v1/score,flesch_kincaid_grade_level/v1/score,ari_grade_level/v1/score,answer_similarity/v1/score,answer_similarity/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,useEffect() is a hook in React that allows you...,44,0.000266,8.3,9.9,4,The output accurately describes what useEffect...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The static keyword in a function means that th...,34,0.000148,13.4,16.0,2,The output provides a correct definition of th...
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python is used to speci...,47,0.000235,10.1,11.1,5,The model's output closely aligns with the pro...
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves the execution of mult...,36,0.000155,21.3,24.4,4,The model's output accurately describes the co...


## Custom LLM-judged metric for professionalism

Create a custom metric that will be used to determine professionalism of the model outputs. Use `make_genai_metric` with a metric definition, grading prompt, grading example, and judge model configuration

In [18]:
from mlflow.metrics.genai import EvaluationExample, make_genai_metric

professionalism_metric = make_genai_metric(
    name="professionalism",
    definition=(
        "Professionalism refers to the use of a formal, respectful, and appropriate style of communication that is tailored to the context and audience. It often involves avoiding overly casual language, slang, or colloquialisms, and instead using clear, concise, and respectful language"
    ),
    grading_prompt=(
        "Professionalism: If the answer is written using a professional tone, below "
        "are the details for different scores: "
        "- Score 1: Language is extremely casual, informal, and may include slang or colloquialisms. Not suitable for professional contexts."
        "- Score 2: Language is casual but generally respectful and avoids strong informality or slang. Acceptable in some informal professional settings."
        "- Score 3: Language is balanced and avoids extreme informality or formality. Suitable for most professional contexts. "
        "- Score 4: Language is noticeably formal, respectful, and avoids casual elements. Appropriate for business or academic settings. "
        "- Score 5: Language is excessively formal, respectful, and avoids casual elements. Appropriate for the most formal settings such as textbooks. "
    ),
    examples=[
        EvaluationExample(
            input="What is MLflow?",
            output=(
                "MLflow is like your friendly neighborhood toolkit for managing your machine learning projects. It helps you track experiments, package your code and models, and collaborate with your team, making the whole ML workflow smoother. It's like your Swiss Army knife for machine learning!"
            ),
            score=2,
            justification=(
                "The response is written in a casual tone. It uses contractions, filler words such as 'like', and exclamation points, which make it sound less professional. "
            ),
        )
    ],
    version="v1",
    model="openai:/gpt-4",
    parameters={"temperature": 0.0},
    grading_context_columns=[],
    aggregations=["mean", "variance", "p90"],
    greater_is_better=True,
)

print(professionalism_metric)

EvaluationMetric(name=professionalism, greater_is_better=True, long_name=professionalism, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's professionalism based on the rubric
justification: Your reasoning about the model's professionalism score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called professionalism based on the input and output.
A definition of professionalism and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before complet

TODO: Try out your new professionalism metric on a sample output to make sure it behaves as you expect

Call `mlflow.evaluate` with your new professionalism metric. 

In [19]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[professionalism_metric],  # use the professionalism metric we created above
    )
print(results.metrics)

2024/01/04 11:22:45 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.6.1.dev0, required: mlflow==2.9.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2024/01/04 11:22:45 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/01/04 11:22:45 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/01/04 11:22:47 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...


  0%|          | 0/1 [00:00<?, ?it/s]

2024/01/04 11:22:51 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: token_count
2024/01/04 11:22:51 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: toxicity
2024/01/04 11:22:51 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: flesch_kincaid_grade_level
2024/01/04 11:22:51 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ari_grade_level
2024/01/04 11:22:51 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: exact_match
2024/01/04 11:22:51 INFO mlflow.models.evaluation.default_evaluator: Evaluating metrics: professionalism


  0%|          | 0/4 [00:00<?, ?it/s]

{'toxicity/v1/mean': 0.00020230028530932032, 'toxicity/v1/variance': 4.11670960581816e-09, 'toxicity/v1/p90': 0.00027421332633821295, 'toxicity/v1/ratio': 0.0, 'flesch_kincaid_grade_level/v1/mean': 14.15, 'flesch_kincaid_grade_level/v1/variance': 6.702500000000001, 'flesch_kincaid_grade_level/v1/p90': 16.970000000000002, 'ari_grade_level/v1/mean': 17.225, 'ari_grade_level/v1/variance': 18.996875000000003, 'ari_grade_level/v1/p90': 22.020000000000003, 'professionalism/v1/mean': 4.0, 'professionalism/v1/variance': 0.0, 'professionalism/v1/p90': 4.0}


In [20]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,toxicity/v1/score,flesch_kincaid_grade_level/v1/score,ari_grade_level/v1/score,professionalism/v1/score,professionalism/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,useEffect() is a hook in React that allows you...,49,0.000191,11.7,12.5,4,The language used in the response is formal an...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The static keyword in a function means that th...,34,0.000146,13.4,16.7,4,The language used in the response is formal an...
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python is used to speci...,65,0.000310,13.0,15.4,4,The language used in the response is formal an...
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves running multiple proc...,33,0.000162,18.5,24.3,4,The language used in the response is formal an...


The professionalism score of the `basic_qa_model` is not very good. Let's try to create a new model that can perform better

Call `mlflow.evaluate()` using the new model. Observe that the professionalism score has increased!

In [21]:
with mlflow.start_run() as run:
    system_prompt = "Answer the following question using extreme formality."
    professional_qa_model = mlflow.openai.log_model(
        model="gpt-4o-mini",
        task=openai.chat.completions,
        name="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        professional_qa_model.model_uri,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[professionalism_metric],
    )
print(results.metrics)

/Users/ann.zhang/anaconda3/envs/mlflow-dev-env/lib/python3.8/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/ann.zhang/anaconda3/envs/mlflow-dev-env/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2024/01/04 11:22:59 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.6.1.dev0, required: mlflow==2.9.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies

  0%|          | 0/1 [00:00<?, ?it/s]

2024/01/04 11:23:17 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: token_count
2024/01/04 11:23:17 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: toxicity
2024/01/04 11:23:17 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: flesch_kincaid_grade_level
2024/01/04 11:23:17 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ari_grade_level
2024/01/04 11:23:17 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: exact_match
2024/01/04 11:23:17 INFO mlflow.models.evaluation.default_evaluator: Evaluating metrics: professionalism


  0%|          | 0/4 [00:00<?, ?it/s]

{'toxicity/v1/mean': 0.00029306011492735706, 'toxicity/v1/variance': 5.9885918086973745e-09, 'toxicity/v1/p90': 0.0003703571535879746, 'toxicity/v1/ratio': 0.0, 'flesch_kincaid_grade_level/v1/mean': 17.1, 'flesch_kincaid_grade_level/v1/variance': 3.7150000000000007, 'flesch_kincaid_grade_level/v1/p90': 18.790000000000003, 'ari_grade_level/v1/mean': 19.575, 'ari_grade_level/v1/variance': 6.421874999999998, 'ari_grade_level/v1/p90': 21.86, 'professionalism/v1/mean': 4.5, 'professionalism/v1/variance': 0.25, 'professionalism/v1/p90': 5.0}


In [22]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,toxicity/v1/score,flesch_kincaid_grade_level/v1/score,ari_grade_level/v1/score,professionalism/v1/score,professionalism/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,The useEffect() function operates by allowing ...,135,0.000183,17.5,19.8,4,The language used in the response is formal an...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...","The static keyword, within the realm of functi...",126,0.000297,14.0,15.5,4,The language used in the response is formal an...
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in the Python programming ...,114,0.000290,19.3,22.4,5,The language used in the response is excessive...
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,"In the realm of computer systems, it is impera...",341,0.000402,17.6,20.6,5,The language used in the response is excessive...
